## Local alignment

#### Smith-Waterman

https://en.wikipedia.org/wiki/Smith–Waterman_algorithm
https://gtuckerkellogg.github.io/pairwise/demo/



In [9]:
import pandas as pd
import numpy as np

In [19]:
x='TGTTACGG'
y='GGTTGACTA'
indel = -2

In [11]:
def createScorMat(match = 1,mismatch = 0):
    """
    Create scoring matrix for nucleotides with a given match and mismatch values
    """
    nts = ['A','C','T','G']

    scoringMatrix = np.zeros((len(nts),len(nts)),)
    np.fill_diagonal(scoringMatrix,match)
    scoringMatrix
    scoringMatrix[scoringMatrix == 0] = mismatch
    scoringMatrix = pd.DataFrame(scoringMatrix,index=nts,columns=nts)
    return scoringMatrix

In [16]:
subMat = createScorMat(3,mismatch = -3)
subMat

,A,C,T,G
A,3.0,-3.0,-3.0,-3.0
C,-3.0,3.0,-3.0,-3.0
T,-3.0,-3.0,3.0,-3.0
G,-3.0,-3.0,-3.0,3.0


['', 'G', 'G', 'T', 'T', 'G', 'A', 'C', 'T', 'A']

,,T,G,T,T,A,C,G,G
,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [106]:
def scoringMatrix_local(x,y,subMat):
    m = len(x) + 1
    n = len(y) + 1
    globMax = (0,0)
    scoringMat=np.zeros((n,m))
    for i in range(1,m):
        #print(x[i-1])
        for j in range(1,n):
            if x[i-1] == y[j-1]:
                #scoringMat[j-1][i-1] + subMat.loc[x[i-1]][y[j-1]]
                scoringMat[j][i] = scoringMat[j-1][i-1] + subMat.loc[x[i-1]][y[j-1]]
            else:
                mismatch = scoringMat[j-1][i-1] + subMat.loc[x[i-1]][y[j-1]]
                left = scoringMat[j-1][i] + indel
                up = scoringMat[j][i-1] + indel
                scoringMat[j][i] = max(mismatch,left,up,0)
            if scoringMat[j][i] >= scoringMat[globMax[1]][globMax[0]]:
                globMax = (i,j)
    return scoringMat,globMax

def vizScoringMat(x,y,scoringMat):
    return pd.DataFrame(scoringMat,index=['']+[_ for _ in y],
             columns=['']+[_ for _ in x])

In [152]:
sm, maxLocs = scoringMatrix_local(x,y,subMat)


('C', 'C')

('GTTGACC', 'GTT-ACC')

In [100]:
sm[j][i]

13.0